In [1]:
import requests
import time
from bs4 import BeautifulSoup
import csv
from collections import defaultdict
import json
import sqlite3
from sqlite3 import Error
from queue import Queue 

In [2]:
# terms_dict = defaultdict(list)
terms_list = []
appDictionary = {}
relatedSearchTerms = {}

In [3]:
def commaSeparated(term):
    result = "";
    if len(term.split(' ')) > 1:
        count = len(term.split(' '));
        i = 0;
        for word in term.split(' '):
            if i > 0:
                result = result + "+";
            result = result + word;
            i = i + 1;
        return result;
    else:
        return term;

In [4]:
#Class to store app details in object format
class AppDetails:
    def __init__(self, title, description, stars, appID, imageSource, developerName):
        self.title = title
        self.description = description
        self.stars = stars
        self.appID = appID
        self.imageSource = imageSource
        self.developerName = developerName
        
    def toString(application):
        return application.title + ", " + application.appID + ", " + application.description + ", " + application.stars + ", " + application.imageSource + ", " + application.developerName

In [5]:
#DB DRIVERS

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return conn
            
            
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
sql_create_appDetailsMainTable_table = """ CREATE TABLE IF NOT EXISTS appDetailsMainTable (
                                        key text NOT NULL,
                                        appIDs text NOT NULL,
                                        relatedSearchTerms text,
                                        source text NOT NULL
                                    ); """

sql_create_appDetails_table = """ CREATE TABLE IF NOT EXISTS appDetails (
                                        appID text NOT NULL,
                                        title text NOT NULL,
                                        description text,
                                        stars text,
                                        imageSource text,
                                        developerName text,
                                        source text
                                    ); """

sql_create_relatedSearchTerms_table = """ CREATE TABLE IF NOT EXISTS relatedSearchTerms (
                                        keyTerm text PRIMARY KEY,
                                        terms text NOT NULL,
                                        source text
                                    ); """

def databaseStartUp():
    # Create connection
    conn = create_connection(r"apkfab.db")

    # Create table
    if conn is not None:
        create_table(conn, sql_create_appDetailsMainTable_table)
        create_table(conn, sql_create_appDetails_table)
        create_table(conn, sql_create_relatedSearchTerms_table)
        print("Database created!")
    else:
        print("Error! cannot create the database connection.")
    
    return conn;
        
        
def insertIntoAppDetailsMainTable(conn, task):
 
    sql = ''' INSERT INTO appDetailsMainTable(key,appIDs,relatedSearchTerms,source)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid

def insertIntoAppDetails(conn, task):
 
    sql = ''' INSERT INTO appDetails(appID, title, description, stars, imageSource, developerName, source)
              VALUES(?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid
     
conn = databaseStartUp()

Database created!


In [6]:
with open('android_terms.csv','rt')as f:
  data = csv.reader(f)
  lineNumber = 0;
  for row in data:
        if lineNumber >= 1:
            key = row[2];
            terms = row[3];
            i = 0;
            for term in terms.split('"'):
                if i%2 == 0:
                    i = i+1;
                    continue
                else:
                    i = i+1;
                    terms_list.append(term)
        else:
            lineNumber = lineNumber + 1;

In [7]:
finalTermsList = []

In [8]:
finalTermsList.clear()
for term in terms_list:
    result = commaSeparated(term)
    finalTermsList.append(result)

In [9]:
len(finalTermsList)

432331

In [ ]:
# WRITTEN FOR TESTING PURPOSE PLEASE IGNORE 

payload = {'q': 'mspy', 't': 'app'}
r = requests.get('https://apk.support/search', params=payload);
print(r.url)
print(r.status_code)

In [ ]:
# TEST
soup = BeautifulSoup(r.text, 'html.parser')
app_table = soup.find_all("h3")
appNamesList = []
for app in app_table:
    appName = app.get_text()
    appNamesList.append(appName)
appDictionary["mspy"] = appNamesList


# TEST Cleanup List
for term in finalTermsList:
    if(term == ""):
        finalTermsList.remove(term)
singleWordSet = set();
for term in finalTermsList:
    if(term.find("+") == -1 and term != ""):
        singleWordSet.add(term)

In [10]:
q = Queue();
wordSet = set();
for term in finalTermsList:
    if(term != ""):
        wordSet.add(term)
print(len(wordSet))
for word in wordSet:
    q.put(word)
print(q.qsize())

1557
1557


In [ ]:
# FOR TEST PURPOSE
singleWordSet = {'cheater', 'cheaters', 'cheating', 'track husband', 'keylogger', 'keyloggers'}

In [ ]:
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word, 't': 'app'};
    r = requests.get('https://apk.support/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    # Get App Names
    names_table = soup.find_all("div", attrs={"class": "it_column"})
    if(len(names_table) == 0):
        print("Skipping " + word)
        continue
    finalList = []
    appIDList = ""
    first = 0
    for name in names_table:
        # Developer Information
        developerPart = name.find_all("div", attrs={"class": "ss_tg"})
        developerPart = developerPart[0].find_all("a")
        developerTag = developerPart[0]['href']
        developerTag = developerTag[10:]
        developerName = developerPart[0].get_text()
        information = name.find_all("a")
        # Title
        titleTag = information[0].find_all("h3")
        title = titleTag[0].get_text()
#         print("Title is " + title)
        # Description
        descriptionTag = information[0].find_all("p")
        description = descriptionTag[0].get_text()
#         print("Description is " + description)
        # Stars
        starsTag = information[0].find_all("div", attrs = {"class" : "stars"})
        starsSpan = starsTag[0].find_all("span")
        stars = starsSpan[0]['title']
        starCount = stars[stars.rindex(' ')+1:]
#         print("Stars is " + starCount)
        # AppID
        appID = information[0]['href']
        appID = appID[4 : ]
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
#         print("AppID:- " + appID)
        # Image Source Link
        imageTag = information[0].find_all("div", attrs={"class" : "seo_img"})
        imageTag = imageTag[0].find_all("img")
        imageSource = imageTag[0]['data-original']
#         print(" ImageSrc:- " + imageSource)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, stars, imageSource, developerName, 'apk.support');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        
    # Suggestion Addition
    suggestionList = soup.find_all("div", attrs={"class": "suggest"})
    suggestionList = suggestionList[0].find_all("li")
    suggestions = []
    suggestionsString = "";
    i = 0
    for suggestion in suggestionList:
        suggestionName = suggestion.get_text()
        if (i != 0):
            suggestionsString = suggestionsString + ","
        suggestionsString = suggestionsString + suggestionName
        i = 1
        suggestions.append(suggestionName)
        modifiedSuggestionName = commaSeparated(suggestionName)
        if(modifiedSuggestionName not in wordSet):
#             print("Added " + modifiedSuggestionName)
            wordSet.add(modifiedSuggestionName)
            q.put(modifiedSuggestionName)
#         else:
#             print("Not Adding " + modifiedSuggestionName)
    
    #Insert Into Main Table
    taskMainTable = (word, appIDList, suggestionsString, 'apk.support');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
    
    
#     appDictionary[word] = finalList
#     relatedSearchTerms[word] = suggestions
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break
    print("Completed " + word)

In [ ]:
#https://apkgk.com/
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'keyword': word}
    r = requests.get('https://apkgk.com/search/', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    appId = soup.find_all("ul", attrs={"class":'topic-wrap'})
    if(len(appId) == 0):
        print("Skipping " + word)
        continue
    appId = appId[0].find_all("a")
    names_table = soup.find_all('div', attrs={"class":'topic-bg'})
    if(len(names_table) == 0):
        print("Skipping " + word)
        continue
    finalList = []
    appIDList = ""
    first = 0
    counter = 0
    for name in names_table:
        appName = name.find_all("div", attrs={"class": "topic-tip-name"})
        appDesc = name.find_all("div", attrs={"class": "topic-tip-description"})
        appSrcMain  = name.find_all("div", attrs={"class": "c-lz-load"})
        imageTag = appSrcMain[0].find_all("img")
        appLink = imageTag[0]['data-src']
        #for img in appSrcMain:
        print(appId[counter]['href'])
        print(appLink)
        print(appName[0].get_text())
        print(appDesc[0].get_text())
        
        appID = appId[counter]['href']
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        imageSource = appLink
        description = appDesc[0].get_text()
        title = appName[0].get_text()
        starCount = "NULL"
        developerName = "NULL"
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'apkgk.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'apkgk.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        
        


In [ ]:
#https://www.malavida.com
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    word = word.replace('+','-')
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    #payload = {'/':word}
    r = requests.get('https://www.malavida.com/en/s/'+word);
    soup = BeautifulSoup(r.text, 'html.parser');
    appDetails = soup.find_all("section", attrs={"class":'app-list'})
    appList = soup.find_all("section", attrs={"class":'app-download'})
    counter = 0
    finalList = []
    appIDList = ""
    first = 0
    notFound = soup.find_all("section", attrs={"class":'not-found'})
    if(len(notFound)>0):
        print("Skipping " + word)
        continue
    print(len(appList))
    for app in appList:
        appSrc = app.find_all("div", attrs={"class":"title"})
        appDesc = app.find_all("p")
        imageSource = app.find_all('img')
        imageSource = imageSource[0]['src']
        appLink = appSrc[0].find_all("a")
        appID = appLink[0]['href']
        title = appLink[0].get_text()
        description = appDesc[0].get_text()
        counter = counter+1
        
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        starCount = "NULL"
        developerName = "NULL"
        print(title +", "+description+", "+imageSource+", "+appID)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'malavida.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'malavida.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        
    
    

In [11]:
#https://apkfab.com/
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    r = requests.get('https://apkfab.com/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    appList = soup.find_all("div", attrs={"class":'list'})
    finalList = []
    appIDList = ""
    first = 0
    for app in appList:
        title = app.find_all("div",attrs={"class":"title"})
        if(len(title)  < 1):
            continue
        title = title[0].get_text()
        rating = app.find_all("span", attrs={"class":"rating"})
        starCount = rating[0].get_text()
        description = "NULL"
        developerName = "NULL"
        imageSource = app.find_all("img")
        imageSource  = imageSource[0]['data-src']
        appID = app.find_all("a")
        appID = appID[0]['href']
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1
        #print(title +", "+description+", "+starCount+", "+imageSource+", "+appID)
        perAppObject = AppDetails(title, description, starCount, appID, imageSource, developerName)
        
        # Insert Into App Table
        taskAppTable = (appID, title, description, starCount, imageSource, developerName, 'apkfab.com');
        insertIntoAppDetails(conn, taskAppTable)
        
        finalList.append(perAppObject)
        #counter=counter+1
    #Insert Into Main Table
    suggestionsString = "NULL"
    taskMainTable = (word, appIDList, suggestionsString, 'apkfab.com');
    insertIntoAppDetailsMainTable(conn, taskMainTable)
        


Starting call+recorder+for+samsung+s8+ 0 with queue length 1556
Starting Automatic+Call+Recorder+(ACR)+Pro 0 with queue length 1555
Starting track+mobile+location+by+number+real 0 with queue length 1554
Starting gps+phone+number+tracker+with+location 0 with queue length 1553
Starting call+recorder+-+hide+app 0 with queue length 1552
Starting synchronize+phone 0 with queue length 1551
Starting control+my+tv+with+my+phone 0 with queue length 1550
Starting call+recorder+2017 0 with queue length 1549
Starting gps+tracking+pro+android 0 with queue length 1548
Starting call+location+track+apps 0 with queue length 1547
Starting sms+from+pc+/+tablet+&+mms+text+messaging+sync+pro 0 with queue length 1546
Starting app+for+tracking+real+time+location+of+other+mobile+no 0 with queue length 1545
Starting app+for+tracking+my+cheating+husband+cell+phone 0 with queue length 1544
Starting find+my+phone+android+app 0 with queue length 1543
Starting girlfriend+mobile+call+and+sms+track 0 with queue lengt

Starting hacker+keyloggers+apk+fb 0 with queue length 1434
Starting free+call+recorder+pro+latest+version 0 with queue length 1433
Starting call+track+location+app 0 with queue length 1432
Starting track+my+phone+for+free 0 with queue length 1431
Starting automatic+call+recorder+pro+2018+callu 0 with queue length 1430
Starting find+my+phone+using+google+account 0 with queue length 1429
Starting Find+My+Phone:+Find+My+Lost+Device 0 with queue length 1428
Starting track+spouse+phone+without+him+knowing 0 with queue length 1427
Starting sms+tracker+free 0 with queue length 1426
Starting call+recorder+app+2018+hd 0 with queue length 1425
Starting track+my+wifes+phone 0 with queue length 1424
Starting hidden+tracker+on+my+husband+cell+phone 0 with queue length 1423
Starting mobile+number+location+track+apps 0 with queue length 1422
Starting kids+phone+control 0 with queue length 1421
Starting track+phone+number+app 0 with queue length 1420
Starting cheater+app+spy+on+another+phone 0 with qu

Starting cheaters+phantom+digital 0 with queue length 1309
Starting call+recorder+app+for+android+oppo 0 with queue length 1308
Starting auto+call+recorder+for+samsung+note+8 0 with queue length 1307
Starting auto+call+recorder+pro+app 0 with queue length 1306
Starting track+your+phone+usage 0 with queue length 1305
Starting cheating+boyfriend+cell+phone+tracker 0 with queue length 1304
Starting track+wife 0 with queue length 1303
Starting track+messages+and+calls+free 0 with queue length 1302
Starting track+my+friend+phone+number+trace+on+map 0 with queue length 1301
Starting automatic+call+recorder+and+hide+app 0 with queue length 1300
Starting track+messages+and+calls 0 with queue length 1299
Starting call+recorder+free+2017 0 with queue length 1298
Starting apps+that+sync+your+text+messages+from+your+phone+to+your+tablet 0 with queue length 1297
Starting phone+call+recording+app+free 0 with queue length 1296
Starting track+children 0 with queue length 1295
Starting verizon+family+l

Starting apps+to+catch+a+cheating+spouse 0 with queue length 1185
Starting find+my+phone+by+serial+number 0 with queue length 1184
Starting app+to+track+location+of+another+mobile 0 with queue length 1183
Starting automatic+call+recorder+hide+apps+pro+callbox 0 with queue length 1182
Starting auto+call+recorder+2017+samsung 0 with queue length 1181
Starting cheating+husband+tracker 0 with queue length 1180
Starting call+recorder+pro+free+download 0 with queue length 1179
Starting Call+Recorder+Automatic++2018 0 with queue length 1178
Starting cheating+boyfriend+tracker 0 with queue length 1177
Starting app+to+track+my+teenagers+phone 0 with queue length 1176
Starting cheating+spouse+app 0 with queue length 1175
Starting find+my+phone+from+google 0 with queue length 1174
Starting gps+tracking+device 0 with queue length 1173
Starting call+recorder+for+android+7.0 0 with queue length 1172
Starting track+location 0 with queue length 1171
Starting hidden+gps+tracker+for+cheating+spouse 0 wi

Starting cheating+spouse+free 0 with queue length 1062
Starting free+sms+tracker+without+other+phone 0 with queue length 1061
Starting how+to+spy+on+husbands+phone+from+my+phone 0 with queue length 1060
Starting phone+number+tracking+gps 0 with queue length 1059
Starting find+phone+location 0 with queue length 1058
Starting cheater+tracker+app 0 with queue length 1057
Starting Auto+Call+Recorder+Pro+2017-18 0 with queue length 1056
Starting cheating+husband+game 0 with queue length 1055
Starting easy+phone+sync 0 with queue length 1054
Starting track+exact+location+of+any+mobile+phone+location+by+cell+number+or+name+of+phone+owner 0 with queue length 1053
Starting my+boyfriend+is+cheating+on+me 0 with queue length 1052
Starting apps+for+spying+on+my+husband 0 with queue length 1051
Starting call+recorder+for+samsung+a7+2017 0 with queue length 1050
Starting automatically+all+call+recorder 0 with queue length 1049
Starting automatic+call+recorder+&+hide+app+pro+-+callboxcall 0 with queu

...

Starting sync+app+for+icloud+photos+to+android+phone 0 with queue length 560
Starting boyfriend+tracker+free 0 with queue length 559
Starting all+automatic+call+recorder+new+apps+free+new+version 0 with queue length 558
Starting a+free+hidden+app+that+can+locate+and+track+a+cell+phone+by+their+text+and+calls 0 with queue length 557
Starting cheating+on+your+girlfriend+games 0 with queue length 556
Starting gps+vehicle+tracking+device 0 with queue length 555
Starting track+a+cell+phone+by+its+number 0 with queue length 554
Starting family+tracker+att 0 with queue length 553
Starting call+and+sms+trackar 0 with queue length 552
Starting find+my+android+phone+google+map+live+location 0 with queue length 551
Starting phone+voice+control+app 0 with queue length 550
Starting phone+number+track+karne+wala+apps 0 with queue length 549
Starting automatic+call+recorder+automatic+call+recording 0 with queue length 548
Starting call+recorder+application 0 with queue length 547
Starting call+record

Starting Call+recorder+automatic 0 with queue length 435
Starting automatic+call+recorder+latest+version+2017 0 with queue length 434
Starting smart+controller+iphone+inoty 0 with queue length 433
Starting whistle+and+find+my+phone 0 with queue length 432
Starting track+unknown+person+exact+home+location+by+phone+number+without+any+app+in+friends+phone 0 with queue length 431
Starting spying+app+for+snapchat 0 with queue length 430
Starting track+location+with+phone+number 0 with queue length 429
Starting call+recording+app+for+samsung+j7 0 with queue length 428
Starting apps+to+find+cheaters 0 with queue length 427
Starting sms+tracker+plus+remote+cell+tracker 0 with queue length 426
Starting free+ringback+tones+apps+for+callers+to+listen+to+when+they+call 0 with queue length 425
Starting find+my+phone+android 0 with queue length 424
Starting track+my+kids+phone 0 with queue length 423
Starting track+cell+phone+location+by+number+without+downloading+app+to+all+phones 0 with queue leng

Starting free+call+and+sms+trackr 0 with queue length 310
Starting best+app+to+spy+on+cheating+spouse+for+free 0 with queue length 309
Starting gps+tracker+phone+number 0 with queue length 308
Starting phone+control 0 with queue length 307
Starting spy+app+for+cheating+spouse 0 with queue length 306
Starting ps4+phone+controller 0 with queue length 305
Starting best+call+recorder+app+2018 0 with queue length 304
Starting find+my+phone+for+android+wear 0 with queue length 303
Starting Video+Call+Recorder+-+Automatic+Call+Recorder+Free 0 with queue length 302
Starting mobile+number+locator+-+track+smart+phone+with+gps 0 with queue length 301
Starting call+recorder+for+samsung+galaxy+s9+plus 0 with queue length 300
Starting simple+easy+and+free+sms+tracker 0 with queue length 299
Starting auto+call+record+app+lock 0 with queue length 298
Starting trace+\u2013+email+tracking 0 with queue length 297
Starting voice+command+app+to+control+mobile+phones+apps+offline+new+latest 0 with queue len

Starting boyfriend+tracker+app+free 0 with queue length 186
Starting free+call+and+sms+trackar 0 with queue length 185
Starting family+locator+and+gps+tracker 0 with queue length 184
Starting automatic+call+recorder+pro+free 0 with queue length 183
Starting track+my+wife+cell+phone+call+details 0 with queue length 182
Starting insighty+-+stalking+and+spying+app+for+instagram 0 with queue length 181
Starting track+lost+phone+by+imei+number 0 with queue length 180
Starting free+app+to+catch+cheaters 0 with queue length 179
Starting hidden+keylogger+for+android 0 with queue length 178
Starting call+recorder+automatically 0 with queue length 177
Starting phone+sync 0 with queue length 176
Starting phone+number+track+location+apps 0 with queue length 175
Starting free+download+latest+version+of+automatic+call+recorder+for+android 0 with queue length 174
Starting heckers+keyloggers 0 with queue length 173
Starting find+friends+location+by+phone+number 0 with queue length 172
Starting track+m

Starting find+my+lost+android+phone 0 with queue length 58
Starting spy+on+my+boyfriend 0 with queue length 57
Starting how+to+track+the+location+of+someone+phone+by+their+mobile+number 0 with queue length 56
Starting phone+number+tracker+with+location+without+gps 0 with queue length 55
Starting call+recorder+\u043e\u0442+appliqato 0 with queue length 54
Starting gps+tracker+my+family 0 with queue length 53
Starting automatic+call+recorder+pro+2018+-+callu 0 with queue length 52
Starting Auto+Call+Recorder+Pro+2018 0 with queue length 51
Starting find+my+android+phone+pebble 0 with queue length 50
Starting call+recorder+app+pro 0 with queue length 49
Starting track+by+email 0 with queue length 48
Starting track+mobile+number+live+location+app 0 with queue length 47
Starting automatic+call+record+download+app 0 with queue length 46
Starting auto+call+recording+application 0 with queue length 45
Starting sms+from+pc+/+tablet+&+mms+text+messaging+sync 0 with queue length 44
Starting mysms

In [ ]:
# Map check Code
for app in appDictionary:
    appList = appDictionary[app]
    print("Entries for " + app)
    for application in appList:
        print(application.toString())
        print("------------------------------->>>")

In [ ]:
# Saving dict to file
with open('apksupport.json', 'w') as f:
    json.dump(appDictionary, f)

with open('relatedSearchTerms_apksupport.json', 'w') as f:
    json.dump(relatedSearchTerms, f)

In [ ]:
SQLite3
dataset package python

upper table -> key and set of app ids
lower table -> app id is primary key and we write to this.

In [ ]:
conn.close()

In [ ]:
# FOR APKCOMBO.COM (ERROR: CLOUDFLARE BLOCKING)
qTrial = Queue()
qTrial.put("cheating+wife")
numberOfTerms = 0
while(qTrial.empty() != True):
    word = qTrial.get()
    time.sleep(5)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    r = requests.get('https://apkcombo.com/search', params=payload);
    soup = BeautifulSoup(r.text, 'html.parser');
    print(soup.prettify())
    # Get App Names
    names_table = soup.find_all("div", attrs={"class": "grid-apps mb-10"})
    

In [ ]:
# FOR APKPURE.COM
qTrial = Queue()
qTrial.put("Starting hidden+tracker+on+my+husband+cell+phone")
qTrial.put("phone+controller+app")
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    checkMore = 1
    firstCheck = 1
    interval = 0
    appIDList = ""
    first = 0
    while(checkMore):
        if(firstCheck == 0):
            interval = interval + 15
        payload = {'q': word,  't': 'app', 'begin': interval};
        payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
        r = requests.get('https://apkpure.com/search-page', params=payload_str);
        soup = BeautifulSoup(r.text, 'html.parser')
        #print(soup.prettify())
        names_table = soup.find_all("dl", attrs={"class": "search-dl"})
        #print(names_table)
        if(len(names_table) == 0):
            print("Skipping " + word)
            break
        for name in names_table:
            dtPart = name.find_all("dt");
            ddPart = name.find_all("dd");
            aPart = dtPart[0].find_all("a");
            appID = aPart[0]['href']
            imagePart = aPart[0].find_all("img");
            imageLink = imagePart[0]['src']
            titlePart = ddPart[0].find_all("p", attrs={"class": "search-title"})
            title = titlePart[0].find("a").get_text();
            starsPart = ddPart[0].find_all("span", attrs={"class": "score"})
            stars = starsPart[0]['title']
            stars = stars.rsplit(' ', 1)[1]
            pParts = ddPart[0].find_all("p")
            developerPart = pParts[1].find_all("a")
            developerName = developerPart[0].get_text()
            if first != 0:
                appIDList = appIDList + ","
            appIDList = appIDList + appID
            first = 1
            
            appTableEntry = (appID, title, 'null', stars, imageLink, developerName, 'apkpure.com');
            insertIntoAppDetails(conn, appTableEntry)
        print("Done")
        firstCheck = 0
        if(len(names_table) == 0):
            checkMore = 0
    appMainTableEntry = (word, appIDList, 'null', 'apkpure.com');
#     print(appMainTableEntry)
    insertIntoAppDetailsMainTable(conn, appMainTableEntry)
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break

In [ ]:
# FOR APK-DL.COM

qTrial = Queue()
qTrial.put("cheating+wife")
qTrial.put("hidden+tracker+on+my+husband+cell+phone")
qTrial.put("phone+controller+app")
numberOfTerms = 0
while(q.empty() != True):
    word = q.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
    r = requests.get('https://apk-dl.com/search', params=payload_str);
    soup = BeautifulSoup(r.text, 'html.parser')
#     print(soup.prettify())
    names_table = soup.find_all("div", attrs={"class": "card no-rationale square-cover apps small"})
    appIDList = ""
    first = 0
    for name in names_table:
        appIDPart = name.find("a", attrs={"class": "card-click-target"})
        appID = appIDPart['href']
#         print(appID)
        imageLinkPart = name.find("img", attrs={"class": "cover-image lazy"})
        imageLink = imageLinkPart['data-original']
#         print(imageLink)
        titlePart = name.find("a", attrs={"class": "title"})
        title = titlePart.get_text()
#         print(title)
        developerNamePart = name.find("a", attrs={"class": "subtitle"})
        developerName = developerNamePart.get_text()
#         print(developerName)
        starsPart = name.find("div", attrs={"class" : "current-rating"})
        stars = starsPart['style']
        stars = stars.rsplit(' ', 1)[1]
        stars = stars[:-2]
        stars = int(stars)/10
#         print(stars)
        
        if first != 0:
            appIDList = appIDList + ","
        appIDList = appIDList + appID
        first = 1

        appTableEntry = (appID, title, 'null', stars, imageLink, developerName, 'apk-dl.com')
        insertIntoAppDetails(conn, appTableEntry)
    
    appMainTableEntry = (word, appIDList, 'null', 'apk-dl.com')
    insertIntoAppDetailsMainTable(conn, appMainTableEntry)
    numberOfTerms = numberOfTerms + 1
    if(numberOfTerms == 5000):
        break
    

In [11]:
# For APKMONK.com

qTrial = Queue()
qTrial.put("cheating+wife")
qTrial.put("hidden+tracker+on+my+husband+cell+phone")
qTrial.put("phone+controller+app")
numberOfTerms = 0
while(qTrial.empty() != True):
    word = qTrial.get()
    time.sleep(1)
    print("Starting " + word + " " + str(numberOfTerms) + " with queue length " + str(q.qsize()))
    payload = {'q': word};
    payload_str = "&".join("%s=%s" % (k,v) for k,v in payload.items())
    r = requests.get('https://www.apkmonk.com/search', params=payload_str);
    soup = BeautifulSoup(r.text, 'html.parser')
    
#     print(soup.prettify())
    names_table = soup.find_all("div", attrs={"class": "col l6 m12 s12 xl4 faceout-row"})
    appIDList = ""
    first = 0
    for name in names_table:
        print(name)
    break

Starting cheating+wife 0 with queue length 1557
